6. การพยากรณ์ยอดขายสินค้าในร้านค้า (Store Item Demand Forecasting)
โจทย์นี้คือตัวแทนของ Multi-Series Time Series Forecasting ซึ่งหมายถึงการพยากรณ์หลายๆ อนุกรมเวลา (ยอดขายของแต่ละสินค้าในแต่ละร้าน) ไปพร้อมๆ กัน เป็นโจทย์ที่เจอบ่อยที่สุดในธุรกิจค้าปลีก

🎯 โจทย์: บริษัทซูเปอร์มาร์เก็ตขนาดใหญ่ต้องการระบบจัดการสินค้าคงคลังที่มีประสิทธิภาพ พวกเขาต้องการโมเดลที่สามารถพยากรณ์ "ยอดขายรายวัน" (sales) ของสินค้าแต่ละชนิด ในแต่ละสาขา ล่วงหน้า 15 วัน เพื่อให้สามารถวางแผนการสั่งซื้อและจัดเก็บสินค้าได้อย่างเหมาะสม ลดปัญหาสินค้าขาดสต็อกและสินค้าล้นสต็อก

🔢 ชุดข้อมูลที่คล้ายกันสำหรับฝึกซ้อม: [Store Item Demand Forecasting Challenge](enge](https://www.google.com/search?q=https://www.kaggle.com/competitions/store-item-demand-forecasting-challenge)


In [1]:
# ติดตั้ง AutoGluon และ library สำหรับดาวน์โหลดข้อมูลจาก Kaggle
# ขั้นตอนนี้อาจใช้เวลา 2-3 นาที
%pip install --upgrade "autogluon[all]" opendatasets


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 8.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of openxlab to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of openxlab to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━

In [7]:
import opendatasets as od
import pandas as pd
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor

# ระบุ URL ของชุดข้อมูลบน Kaggle
dataset_url = 'https://www.kaggle.com/c/demand-forecasting-kernels-only'

# ดาวน์โหลดข้อมูล (ระบบจะถามหา Kaggle Username และ Key)
od.download(dataset_url)

# กำหนด Path ของไฟล์ข้อมูล
data_dir = '/content/demand-forecasting-kernels-only'

# โหลดข้อมูล train และ test ด้วย pandas
train_df = pd.read_csv(f'{data_dir}/train.csv')
test_df = pd.read_csv(f'{data_dir}/test.csv')

# แสดงขนาดและตัวอย่างข้อมูลเพื่อตรวจสอบความถูกต้อง
print(f"ขนาดข้อมูล Train: {train_df.shape}")
print(f"ขนาดข้อมูล Test:  {test_df.shape}")
print("\nตัวอย่างข้อมูล Train 5 แถวแรก:")
train_df.head()


Skipping, found downloaded files in "./demand-forecasting-kernels-only" (use force=True to force download)
ขนาดข้อมูล Train: (913000, 4)
ขนาดข้อมูล Test:  (45000, 4)

ตัวอย่างข้อมูล Train 5 แถวแรก:


,date,store,item,sales
0,2013-01-01,1,1,13
1,2013-01-02,1,1,11
2,2013-01-03,1,1,14
3,2013-01-04,1,1,13
4,2013-01-05,1,1,10


In [8]:
# --- จัดการข้อมูล Train ---
# แปลงคอลัมน์ date เป็น datetime
train_df['date'] = pd.to_datetime(train_df['date'])

# สร้างคอลัมน์ item_id ที่เป็นเอกลักษณ์สำหรับแต่ละซีรีส์ (Store + Item)
train_df['item_id'] = train_df['store'].astype(str) + '_' + train_df['item'].astype(str)

# สร้าง TimeSeriesDataFrame
# ระบุ item_id และ timestamp ให้ถูกต้อง
train_data = TimeSeriesDataFrame.from_data_frame(
    train_df,
    id_column="item_id",
    timestamp_column="date"
)

print("ตัวอย่าง TimeSeriesDataFrame สำหรับ Train:")
train_data.head()


ตัวอย่าง TimeSeriesDataFrame สำหรับ Train:


store  item  sales
item_id timestamp                     
1_1     2013-01-01      1     1     13
        2013-01-02      1     1     11
        2013-01-03      1     1     14
        2013-01-04      1     1     13
        2013-01-05      1     1     10

In [9]:
# กำหนดค่าต่างๆ สำหรับ Predictor
prediction_length = 90  # พยากรณ์ล่วงหน้า 90 วันตามโจทย์
eval_metric = "sMAPE"   # ใช้เกณฑ์การประเมินของการแข่งขัน
time_limit = 600        # กำหนดเวลาฝึก 10 นาที (อาจปรับเพิ่มได้เพื่อคุณภาพที่ดีขึ้น)

# สร้าง Predictor
predictor = TimeSeriesPredictor(
    prediction_length=prediction_length,
    path="ag_models_demand",
    target="sales",
    eval_metric=eval_metric
)

# เริ่มฝึกโมเดล
predictor.fit(
    train_data,
    presets="medium_quality", # ใช้ medium_quality เพื่อความรวดเร็ว แต่ยังได้โมเดลที่ดี
    time_limit=time_limit,
)


Beginning AutoGluon training... Time limit = 600s
AutoGluon will save models to '/content/ag_models_demand'
=================== System Info ===================
AutoGluon Version:  1.3.1
Python Version:     3.11.13
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Sun Mar 30 16:01:29 UTC 2025
CPU Count:          2
GPU Count:          0
Memory Avail:       11.11 GB / 12.67 GB (87.7%)
Disk Space Avail:   65.00 GB / 107.72 GB (60.3%)
Setting presets to: medium_quality

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': SMAPE,
 'hyperparameters': 'light',
 'known_covariates_names': [],
 'num_val_windows': 1,
 'prediction_length': 90,
 'quantile_levels': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
 'random_seed': 123,
 'refit_every_n_windows': 1,
 'refit_full': False,
 'skip_model_selection': False,
 'target': 'sales',
 'time_limit': 600,
 'verbosity': 2}

Inferred time series frequency: 'D'
Provided train_data has 913000 rows, 

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/191M [00:00<?, ?B/s]

	Time limit exceeded... Skipping Chronos[bolt_small].
Training timeseries model TemporalFusionTransformer. Training for up to 81.5s of the 163.0s of remaining time.
	-0.1758       = Validation score (-SMAPE)
	77.99   s     = Training runtime
	2.81    s     = Validation (prediction) runtime
Fitting simple weighted ensemble.
	Ensemble weights: {'DirectTabular': 0.89, 'RecursiveTabular': 0.11}
	-0.1238       = Validation score (-SMAPE)
	4.65    s     = Training runtime
	6.91    s     = Validation (prediction) runtime
Training complete. Models trained: ['Naive', 'SeasonalNaive', 'RecursiveTabular', 'DirectTabular', 'TemporalFusionTransformer', 'WeightedEnsemble']
Total runtime: 515.48 s
Best model: WeightedEnsemble
Best model score: -0.1238


In [10]:
# แสดง Leaderboard ของโมเดลทั้งหมด
# สำหรับ sMAPE ค่ายิ่งน้อยยิ่งดี
predictor.leaderboard()


,model,score_val,pred_time_val,fit_time_marginal,fit_order
0,WeightedEnsemble,-0.123844,6.912214,4.649390,6
1,DirectTabular,-0.124286,3.372830,56.396564,4
2,RecursiveTabular,-0.146719,3.539384,78.010504,3
3,TemporalFusionTransformer,-0.175798,2.805390,77.989599,5
4,SeasonalNaive,-0.222704,1.397095,1.223918,2
5,Naive,-0.274699,2.177728,1.431261,1


In [18]:
# พยากรณ์อนาคตจากข้อมูล train ทั้งหมด
# AutoGluon จะพยากรณ์ 90 วันข้างหน้าสำหรับทุก item_id
predictions = predictor.predict(train_data)

# แสดงตัวอย่างผลการพยากรณ์
print("ตัวอย่างผลการพยากรณ์:")
display(predictions.head())

# --- จัดรูปแบบสำหรับไฟล์ Submission ---
# รีเซ็ต index เพื่อให้ timestamp (date) กลายเป็นคอลัมน์ปกติ
predictions_df = predictions.reset_index()

# Rename the timestamp column to 'date'
predictions_df.rename(columns={'timestamp': 'date'}, inplace=True)

# Convert 'date' column in test_df to datetime
test_df['date'] = pd.to_datetime(test_df['date'])

# สร้างคอลัมน์ store และ item กลับคืนมาจาก item_id
predictions_df[['store', 'item']] = predictions_df['item_id'].str.split('_', expand=True)
predictions_df['store'] = predictions_df['store'].astype(int)
predictions_df['item'] = predictions_df['item'].astype(int)

# เชื่อม (merge) ผลการทำนายเข้ากับ test_df เพื่อให้ได้ 'id' ที่ถูกต้อง
submission = test_df.merge(
    predictions_df,
    on=['date', 'store', 'item'],
    how='left'
)

# เลือกเฉพาะคอลัมน์ id และเปลี่ยนชื่อคอลัมน์ mean เป็น sales
submission = submission[['id', 'mean']]
submission.rename(columns={'mean': 'sales'}, inplace=True)

# ค่าพยากรณ์อาจเป็นทศนิยมและอาจติดลบได้ ควรจัดการให้เหมาะสม
submission['sales'] = submission['sales'].apply(lambda x: max(0, x)) # ทำให้ค่าไม่ติดลบ
submission['sales'] = submission['sales'].round().astype(int) # ปัดเป็นจำนวนเต็ม

# แสดงตัวอย่างไฟล์ submission
print("\nตัวอย่างไฟล์ Submission:")
display(submission.head())

# บันทึกเป็นไฟล์ CSV
submission.to_csv('submission_demand.csv', index=False)

print("\nสร้างไฟล์ submission_demand.csv เรียบร้อยแล้ว!")

Model not specified in predict, will default to the model with the best validation score: WeightedEnsemble


ตัวอย่างผลการพยากรณ์:


mean       0.1       0.2        0.3        0.4  \
item_id timestamp                                                         
1_1     2018-01-01  12.231527  6.066202  8.182632   9.708726  11.012718   
        2018-01-02  13.745566  5.026480  8.019563  10.177786  12.021909   
        2018-01-03  14.542539  3.863884  7.529647  10.172920  12.431499   
        2018-01-04  15.491729  3.161079  7.393938  10.446127  13.054110   
        2018-01-05  16.302280  2.516195  7.248675  10.661126  13.576939   

                          0.5        0.6        0.7        0.8        0.9  
item_id timestamp                                                          
1_1     2018-01-01  12.231527  13.450336  14.754328  16.280422  18.396852  
        2018-01-02  13.745566  15.469222  17.313345  19.471568  22.464651  
        2018-01-03  14.542539  16.653579  18.912158  21.555432  25.221195  
        2018-01-04  15.491729  17.929348  20.537330  23.589519  27.822378  
        2018-01-05  16.302280  19.027621  21.943434  25.355885  30.088365


ตัวอย่างไฟล์ Submission:


,id,sales
0,0,12
1,1,14
2,2,15
3,3,15
4,4,16



สร้างไฟล์ submission_demand.csv เรียบร้อยแล้ว!


In [21]:
predictions_df

,item_id,date,mean,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,store,item
0,1_1,2018-01-01,12.231527,6.066202,8.182632,9.708726,11.012718,12.231527,13.450336,14.754328,16.280422,18.396852,1,1
1,1_1,2018-01-02,13.745566,5.026480,8.019563,10.177786,12.021909,13.745566,15.469222,17.313345,19.471568,22.464651,1,1
2,1_1,2018-01-03,14.542539,3.863884,7.529647,10.172920,12.431499,14.542539,16.653579,18.912158,21.555432,25.221195,1,1
3,1_1,2018-01-04,15.491729,3.161079,7.393938,10.446127,13.054110,15.491729,17.929348,20.537330,23.589519,27.822378,1,1
4,1_1,2018-01-05,16.302280,2.516195,7.248675,10.661126,13.576939,16.302280,19.027621,21.943434,25.355885,30.088365,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44995,10_50,2018-03-27,69.280660,-36.998843,-0.515272,25.791949,48.270499,69.280660,90.290821,112.769371,139.076592,175.560163,10,50
44996,10_50,2018-03-28,68.390914,-38.504707,-1.809636,24.650092,47.258954,68.390914,89.522874,112.131736,138.591464,175.286535,10,50
44997,10_50,2018-03-29,75.156984,-32.351224,4.554136,31.165497,53.903923,75.156984,96.410045,119.148472,145.759833,182.665193,10,50
44998,10_50,2018-03-30,77.234688,-30.882637,6.231820,32.993955,55.861212,77.234688,98.608163,121.475421,148.237556,185.352013,10,50


In [20]:
test_df

,id,date,store,item
0,0,2018-01-01,1,1
1,1,2018-01-02,1,1
2,2,2018-01-03,1,1
3,3,2018-01-04,1,1
4,4,2018-01-05,1,1
...,...,...,...,...
44995,44995,2018-03-27,10,50
44996,44996,2018-03-28,10,50
44997,44997,2018-03-29,10,50
44998,44998,2018-03-30,10,50


In [19]:
test_df['date']

,date
0,2018-01-01
1,2018-01-02
2,2018-01-03
3,2018-01-04
4,2018-01-05
...,...
44995,2018-03-27
44996,2018-03-28
44997,2018-03-29
44998,2018-03-30


In [17]:
submission

,id,sales
0,0,12
1,1,14
2,2,15
3,3,15
4,4,16
...,...,...
44995,44995,69
44996,44996,68
44997,44997,75
44998,44998,77
